<a href="https://colab.research.google.com/github/Pythonash/Kaggle-Kor-/blob/Brain/%ED%99%95%EB%A5%A0%EC%A0%81%20%EB%AA%A8%EC%88%98%20%ED%83%90%EC%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이 노트북은 Pythonash에 의해 작성되었습니다.

이 노트북의 목적은 여러 파라미터들이 변화할 때 손실 결과 데이터를 수집하는 것에 있습니다.

여러 많은 파라미터들의 조합으로, 최적의 파라미터나 어떻게 당신의 모델을 최적화 할 수 있을지 찾을 수 있을 것 입니다.

그럼 시작해 볼까요?

# 필요한 라이브러리들을 불러와 줍니다.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
# import ubiquant
from sklearn.model_selection import KFold
import random
import time

start = time.time()

# 데이터셋을 불러오고, 이를 핸들링 합니다.

- 데이터를 불러오고 핸들링하는 자세한 내용은 저의 이전 노트북인 [End to end simple and powerful DNN with LeakyReLU](https://www.kaggle.com/pythonash/end-to-end-simple-and-powerful-dnn-with-leakyrelu)에 설명되어 있습니다.

- 만약 이에 대해 더 많은 정보가 필요하시다면, 위의 링크를 눌러 확인해 주세요.


## 주의

이 노트북은 과정이 약간 다릅니다.

전의 노트북에서는 "investment_id"와 "표준화"를 사용하였지만, 이번 노트북 에서는 "investment_id"를 사용하지 않으며, 표준화 대신 정규화(Min-Max)를 사용합니다.

In [ ]:
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
df

In [ ]:
f_col = df.drop(['row_id','time_id','investment_id','target'],axis=1).columns
f_col

In [ ]:
scaler = MinMaxScaler()
scaler.fit(pd.DataFrame(df[f_col]))

In [ ]:
def make_dataset(df):
    f_df = df[f_col]
    scaled_f = scaler.transform(pd.DataFrame(f_df))
    data_x = pd.DataFrame(scaled_f)
    data_x.columns = f_df.columns
    del f_df
    data_x = data_x.astype('float16')
    return data_x

In [ ]:
# df=df.astype('float16')
df_x = make_dataset(df)
df_x

In [ ]:
df_y = pd.DataFrame(df['target'])
df_y

In [ ]:
del df

# 확률적 모델 구조

- 저는 모델을 랜덤하게 구성하였습니다. 즉, 뉴런의 수나 드롭아웃 비율이나 학습률이 매번 다르게 할당될 것 입니다.

- LeakyReLU 대신 PReLU를 사용합니다.

- 매 반복마다 랜덤한 뉴런과 드롭아웃 비율, 그리고 학습률을 얻으실 겁니다.

- 그렇게 함으로써, 파라미터가 변할때 마다 손실 값에 미치는 영향에 대한 기록 데이터를 얻으실 수 있습니다.

모델의 구조는 원하시는 만큼 바꿔서 해보실 수 있습니다.


In [ ]:
def pythonash_model():
    neurons = random.randint(16, 513)
    drop_rate = random.randint(1,6)/10
    lr_rate = random.uniform(0.0005, 0.007)
    
    inputs_ = tf.keras.Input(shape = [df_x.shape[1]])
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(inputs_)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(leaky)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(leaky)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(leaky)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(leaky)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    drop = tf.keras.layers.Dropout(drop_rate)(leaky)
    
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(drop)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    
    x = tf.keras.layers.Dense(neurons, kernel_initializer = 'he_normal')(leaky)
    batch = tf.keras.layers.BatchNormalization()(x)
    leaky = tf.keras.layers.PReLU()(batch)
    drop = tf.keras.layers.Dropout(drop_rate)(leaky)
    
    outputs_ = tf.keras.layers.Dense(1)(drop)
    
    model = tf.keras.Model(inputs = inputs_, outputs = outputs_)
    
    rmse = tf.keras.metrics.RootMeanSquaredError()

    learning_sch = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = lr_rate,
    decay_steps = 10000,
    decay_rate = 0.97)
    
    adam = tf.keras.optimizers.Adam(learning_rate = learning_sch)
    
    model.compile(loss = 'mse', metrics = rmse, optimizer = adam)
    opt_name = str(model.optimizer).split('.')[3].split()[0]
    print('Current set is \n neurons: {0},\n Drop rate: {1}, \n learning_rate: {2}'.format(neurons, drop_rate, lr_rate))
    
    return neurons, drop_rate, lr_rate, model

# 시뮬레이션 로그 저잠 하기

매 반복마다, 손실 값이 계산되는데 이 코드는 자동으로 그것을 저장해 줍니다.


In [ ]:
simulation_log = []
num_iter = 1
for iteration in np.arange(1,100):
    end = time.time()#
    if round(end - start,0) /3600 < 5:#
        print('Current running time: {} sec.'.format(round(end - start,0)))
        num_fold = 1
        kfold_generator = KFold(n_splits =5, shuffle=True)
        callbacks = tf.keras.callbacks.ModelCheckpoint('pythonash_model.h5', save_best_only = True)
        neurons, drop_rate, lr_rate, model = pythonash_model()
        fold_model = model.save('fold_model.h5')
        del fold_model
        del model
        for train_index, val_index in kfold_generator.split(df_x, df_y):
            fold_model = tf.keras.models.load_model('fold_model.h5')
            # Split training dataset.
            train_x, train_y = df_x.iloc[train_index], df_y.iloc[train_index]
            # Split validation dataset.
            val_x, val_y = df_x.iloc[val_index], df_y.iloc[val_index]
            # Make tensor dataset.
            tf_train = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(2022).batch(1024, drop_remainder=True).prefetch(1)
            tf_val = tf.data.Dataset.from_tensor_slices((val_x, val_y)).shuffle(2022).batch(1024, drop_remainder=True).prefetch(1)
            # Load model
            ###############################################################################################        
            print('======================================Fold %d Start!======================================='%num_fold)
            fit_history = fold_model.fit(tf_train, callbacks = callbacks, epochs = 5, #### change the epochs into more numbers.
                     validation_data = (tf_val), shuffle=True, verbose = 1)
            min_loss = np.array(fit_history.history['val_loss']).min()
            print('===========================================================================================')
            print('Model achieves %f in validation set.' %min_loss)
            print('===========================================================================================')
            simulation_log.append([num_iter, num_fold, neurons, drop_rate, lr_rate, min_loss])
            log_df = pd.DataFrame(simulation_log)
            log_df.columns = ['num_iter','num_fold','neurons', 'drop_rate', 'lr_rate', 'min_loss']
            print(log_df)
            log_df.to_csv('./Parameter finder log.csv', encoding = 'utf-8-sig', index = False)
            print('===========================================================================================')
            # Delete tensor dataset and model for avoiding memory exploring.
            del tf_train
            del tf_val
            del fit_history
            del fold_model
            num_fold += 1
    else:
        print('Memory using time is over.')
        break
#     del model
    del neurons
    del drop_rate
    del lr_rate
    del min_loss
    print('%d iteraion is over.' %num_iter)
    print('===========================================================================================')
    num_iter+=1
    